In [45]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from glob import glob
import time
import random
import requests
import os
import glob
import re
import csv
from html.parser import HTMLParser


class GRB_list_text_crawl():

    def __init__(self, driver_path, download_path, crawler_year):
        self.url_to_crawl = 'https://www.grb.gov.tw/'
        self.driver_path = driver_path
        self.regi_path = regi_path
        self.SaveFile_Path = SaveFile_Path
        self.SaveFile_Name = SaveFile_Name
        self.proj = proj
        self.proj_aut = proj_aut
        self.proj_year = proj_year

    def start(self):
        self.browser = webdriver.Chrome(self.driver_path)
        self.browser.maximize_window()

    def get_into_page(self, url):
        self.browser.get(url)
        time.sleep(5)
        #先點一次查詢，進到搜尋主頁面
        self.browser.find_element(By.CLASS_NAME, "butsearch").click()
        time.sleep(5)
        self.browser.find_element(By.CSS_SELECTOR, ".m-t-80").clear()
        time.sleep(5)

    def search_work(self, unit):
        self.browser.find_element(
            By.CSS_SELECTOR, ".m-t-80").send_keys(proj.iloc[self.i][:unit] +
                                                  " + " + proj_aut[self.i])
        time.sleep(4)
        self.browser.find_element(By.CSS_SELECTOR,
                                  ".w-160 > .width-60:nth-child(1)").click()
        time.sleep(2)
        Select(
            self.browser.find_element(
                By.CSS_SELECTOR,
                ".w-160 > .width-60:nth-child(1)")).select_by_value(
                    str(proj_year[self.i]))
        time.sleep(4)
        self.browser.find_element(
            By.XPATH,
            "//*[@id='aiSearch2']/form/div[4]/table[1]/tbody/tr/td[4]/select[2]"
        ).click()
        time.sleep(2)
        Select(
            self.browser.find_element(
                By.XPATH,
                "//*[@id='aiSearch2']/form/div[4]/table[1]/tbody/tr/td[4]/select[2]"
            )).select_by_value(str(proj_year[self.i]))
        time.sleep(2)
        self.browser.find_element(By.CSS_SELECTOR, ".butsearch").click()
        WebDriverWait(self.browser, 30).until(  #等待網頁讀取 直到再次出現 "搜尋框"
            EC.presence_of_element_located((By.CLASS_NAME, "butsearch")))
        time.sleep(3)
        self.browser.execute_script("window.scrollTo(0,413)")
        WebDriverWait(self.browser, 30).until(  #等待網頁讀取 直到再次出現 "搜尋框"
            EC.presence_of_element_located((By.CLASS_NAME, "butsearch")))

    def strip_tags(self, html):
        html = html.strip()
        html = html.strip('\n')
        result = []
        parser = HTMLParser()
        parser.handle_data = result.append
        parser.feed(html)
        parser.close()
        result = ''.join(result)
        result = result.replace('\n', '')
        return result

    def download_file(self, soup):
        result_list = []
        try:  #報告詳目檢查
            time.sleep(5.2)
            self.browser.find_element_by_class_name('tabBg2').click()
            time.sleep(5.2)
            ele_find_all = soup.find_all('span', {'class': 'conalltxt'})
            ele_find_all2 = soup.find_all('span', {'class': 'contxt'})
            aut = self.strip_tags(str(ele_find_all[10]))
            #str(ele_find_all[10]).replace('<span _ngcontent-c5="" class="conalltxt">','').replace('\xa0</span>', '')
            proj_num = self.strip_tags(str(ele_find_all2[2])).strip()
            #str(ele_find_all2[2]).replace('<span _ngcontent-c5="" class="contxt">','').replace('</span>', '')
            agency = soup.find("span",
                               string="執行機構").find_next_sibling("span").text
            en_title = soup.find('span', {'class': 'planTitleen'}).text
            FOS = soup.find("span",
                            string="研究領域").find_next_sibling("span").text
            proj_cost = soup.find("span",
                                  string="研究經費").find_next_sibling("span").text
            starend = soup.find(
                "span", string="研究期間").find_next_sibling("span").text.replace(
                    '~', '/').split('/')
            ch_keyword = soup.find(
                "span", string="中文關鍵字").find_next_sibling("span").text
            eng_keyword = soup.find(
                "span", string="英文關鍵字").find_next_sibling("span").text
            try:
                ch_abst = soup.find(
                    "span", string="中文計畫概述").find_next_sibling("span").text
            except:
                ch_abst = ''
            try:
                eng_abst = soup.find(
                    "span", string="英文計畫概述").find_next_sibling("span").text
            except:
                eng_abst = ''
            proposal_report = '報告詳目'
            result_list.append([
                aut, proj_num, agency, self.proj.iloc[self.i], en_title, FOS,
                proj_cost, starend, ch_keyword, eng_keyword, ch_abst, eng_abst,
                proposal_report
            ])
        except:
            try:
                time.sleep(5.2)
                soup = BeautifulSoup(self.browser.page_source, 'lxml')  #擷取每頁頁面
                ele_find_all = soup.find_all('span', {'class': 'conalltxt'})
                aut = self.strip_tags(
                    str(ele_find_all[0]).replace(
                        '</span><span _ngcontent-c5="" class="p-r-10">', ';'))
                #str(ele_find_all[0]).replace('</span><span _ngcontent-c5="" class="p-r-10">', ';').replace('<span _ngcontent-c5="" class="conalltxt"><!-- --><span _ngcontent-c5="" class="p-r-10">','').replace('</span></span>', '')
                proj_num = soup.find(
                    "span",
                    string="計畫編號").find_next_sibling("span").text.strip()
                agency = soup.find(
                    "span", string="執行機構").find_next_sibling("span").text
                en_title = soup.find('span', {'class': 'planTitleen'}).text
                FOS = soup.find("span",
                                string="研究領域").find_next_sibling("span").text
                proj_cost = soup.find(
                    "span", string="研究經費").find_next_sibling("span").text
                starend = soup.find(
                    "span",
                    string="研究期間").find_next_sibling("span").text.replace(
                        '~', '/').split('/')
                ch_keyword = soup.find(
                    "span", string="中文關鍵字").find_next_sibling("span").text
                eng_keyword = soup.find(
                    "span", string="英文關鍵字").find_next_sibling("span").text
                try:
                    ch_abst = soup.find(
                        "span", string="中文計畫概述").find_next_sibling("span").text
                except:
                    ch_abst = ''
                try:
                    eng_abst = soup.find(
                        "span", string="英文計畫概述").find_next_sibling("span").text
                except:
                    eng_abst = ''
                proposal_report = '計畫詳目'
            except:
                aut = proj_aut[self.i]
                proj_num = ''
                agency = ''
                en_title = ''
                FOS = ''
                proj_cost = ''
                starend = ''
                ch_keyword = ''
                eng_keyword = ''
                ch_abst = ''
                eng_abst = ''
                proposal_report = '程式在GRB查無資料，請手動查詢。'
            result_list.append([
                aut, proj_num, agency, self.proj.iloc[self.i], en_title, FOS,
                proj_cost, starend, ch_keyword, eng_keyword, ch_abst, eng_abst,
                proposal_report
            ])
        return result_list

    def roll_page(self):  #翻頁並進入search_wor和download_file的步驟
        res_all = pd.DataFrame()
        proj_all = len(projTT)
        lst1 = list(range(0, proj_all))
        res_process = pd.DataFrame()
        self.error_list = []
        for self.i in lst1:
            soup = BeautifulSoup(self.browser.page_source, 'lxml')  #擷取每頁頁面
            self.search_work(500)
            window_before = self.browser.window_handles[0]
            time.sleep(4)
            try:
                self.browser.find_element(By.CSS_SELECTOR,
                                          ".highlight:nth-child(1)").click()
                WebDriverWait(self.browser, 30).until(  #等待網頁讀取 直到再次出現 "搜尋框"
                    EC.presence_of_element_located((By.ID, "maincontent")))
                window_after = self.browser.window_handles[1]
                WebDriverWait(self.browser, 30).until(  #等待網頁讀取 直到再次出現 "搜尋框"
                    EC.presence_of_element_located((By.ID, "resultR")))
                #切換到新視窗
                self.browser.switch_to.window(window_after)
                soup = BeautifulSoup(self.browser.page_source, 'lxml')  #擷取每頁頁面
                time.sleep(4)
                result_list = self.download_file(soup)
                self.browser.close()
                #切換到舊視窗
                self.browser.switch_to.window(window_before)
                self.browser.find_element(By.CSS_SELECTOR, ".m-t-80").clear()
                time.sleep(5)
                result_df = pd.DataFrame(result_list)
                res_process = res_process.append(result_df)
            except:
                try:
                    self.browser.find_element(By.CSS_SELECTOR,
                                              ".m-t-80").clear()
                    self.search_work(28)
                    time.sleep(4)
                    window_before = self.browser.window_handles[0]
                    self.browser.find_element(
                        By.CSS_SELECTOR, ".highlight:nth-child(1)").click()
                    WebDriverWait(self.browser,
                                  30).until(  #等待網頁讀取 直到再次出現 "搜尋框"
                                      EC.presence_of_element_located(
                                          (By.ID, "maincontent")))
                    window_after = self.browser.window_handles[1]
                    WebDriverWait(self.browser,
                                  30).until(  #等待網頁讀取 直到再次出現 "搜尋框"
                                      EC.presence_of_element_located(
                                          (By.ID, "resultR")))
                    #切換到新視窗
                    self.browser.switch_to.window(window_after)
                    soup = BeautifulSoup(self.browser.page_source,
                                         'lxml')  #擷取每頁頁面
                    time.sleep(4)
                    result_list = self.download_file(soup)
                    self.browser.close()
                    #切換到舊視窗
                    self.browser.switch_to.window(window_before)
                    self.browser.find_element(By.CSS_SELECTOR,
                                              ".m-t-80").clear()
                    time.sleep(5)
                    result_df = pd.DataFrame(result_list)
                    res_process = res_process.append(result_df)
                except:
                    error = '程式在GRB查無資料，請手動查詢。'
                    aut = proj_aut[self.i]
                    proj_num = ''
                    agency = ''
                    en_title = ''
                    FOS = ''
                    proj_cost = ''
                    starend = ''
                    ch_keyword = ''
                    eng_keyword = ''
                    ch_abst = ''
                    eng_abst = ''
                    self.error_list.append([
                    aut, proj_num, agency, self.proj.iloc[self.i], en_title, FOS,
                    proj_cost, starend, ch_keyword, eng_keyword, ch_abst, eng_abst,
                    error
                    ])
                    result_df = pd.DataFrame(self.error_list)
                    res_process = res_process.append(result_df)
                    self.browser.refresh()
                    self.browser.find_element(By.CSS_SELECTOR,
                                              ".m-t-80").clear()
        res_all = res_all.append(res_process)
        res_all.columns = [
            '計畫主持人', '計畫編號', '執行機構', '中文計畫名稱', '英文計畫名稱', '研究領域', '研究經費',
            '研究期間', '中文關鍵字', '英文關鍵字', '中文摘要', '英文摘要', '網站依據'
        ]
        res_all = res_all.drop_duplicates(subset='計畫名稱',
                      keep='first',
                      inplace=False,
                      ignore_index=False)
        try:
            res_all.to_excel(SaveFile_Path + '\\' + SaveFile_Name + "_" + "_" +
                             time.strftime("%m%d") + "_" +
                             time.strftime("%H%M") + ".xlsx",
                             encoding="utf_8_sig",
                             index=False)  #儲存結果
            print("存檔成功")  #最終檔名會加上 1.月份 2.日期 3.時間(時分)
        except:
            print("存檔失敗，請檢察結果檔是否正在開啟，或處於無法被寫入的狀態。")
        self.error_check()

    def error_check(self):
        try:
            errof_df = pd.DataFrame(self.error_list)
            error_check = '程式在GRB查無資料，請手動查詢。' in errof_df.iloc[:,12].values
            if error_check == True:
                print('有資料缺失')
                print('共有' + str(len(self.error_list)) + '筆缺失資料')
        except:
            print('無資料缺失')

    def close_driver(self):
        self.browser.quit()
        print('成功爬取GRB清冊')

    def step(self):
        self.start()
        self.get_into_page(self.url_to_crawl)
        print('正在爬取GRB清冊')
        self.roll_page()
        self.close_driver()


if __name__ == '__main__':
    driver_path = r'C:\Users\user\Desktop\py\Driver\chromedriver.exe'  #chromedriver位置
    regi_path = r"C:\Users\user\Desktop\py\merge\108調查計畫名冊_全司1081122.xlsx"  #科技部名冊路徑 ★請注意前面必須有小r★
    SaveFile_Path = r'C:\Users\user\Desktop\py\TEST'  #要儲存檔案的路徑
    SaveFile_Name = r'GRB_list_text_crawl'  #儲存的檔名，可以不用更動
    projTT = pd.read_excel(regi_path)
    proj = projTT['計畫名稱']
    proj_aut = projTT['計畫主持人']
    proj_year = projTT['補'].astype('Int64')
    go = GRB_list_text_crawl(driver_path, SaveFile_Path, regi_path)

    go.step()

C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:34: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.browser = webdriver.Chrome(self.driver_path)


正在爬取GRB清冊


C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:95: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  self.browser.find_element_by_class_name('tabBg2').click()
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:221: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:273: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:95: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  self.browser.find_element_by_class_name('tabBg2').click()
C:\U

C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:95: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  self.browser.find_element_by_class_name('tabBg2').click()
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:221: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:95: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  self.browser.find_element_by_class_name('tabBg2').click()
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:221: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\U

C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:221: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:95: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  self.browser.find_element_by_class_name('tabBg2').click()
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:221: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:95: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  self.browser.find_element_by_class_name('tabBg2').click()
C:\U

...

C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:221: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:95: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  self.browser.find_element_by_class_name('tabBg2').click()
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:221: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:95: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  self.browser.find_element_by_class_name('tabBg2').click()
C:\U

C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:95: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  self.browser.find_element_by_class_name('tabBg2').click()
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:221: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:95: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  self.browser.find_element_by_class_name('tabBg2').click()
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:221: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\U

C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:221: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:95: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  self.browser.find_element_by_class_name('tabBg2').click()
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:221: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:95: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  self.browser.find_element_by_class_name('tabBg2').click()
C:\U

C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:95: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  self.browser.find_element_by_class_name('tabBg2').click()
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:221: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:95: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  self.browser.find_element_by_class_name('tabBg2').click()
C:\Users\user\AppData\Local\Temp\ipykernel_22096\1669154651.py:221: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\U

存檔成功
有資料缺失
共有2筆缺失資料
成功爬取GRB清冊


In [ ]:
regi_path = r"C:\Users\user\Desktop\py\TEST\GRB_list_text_crawl__0816_1350.xlsx"  #科技部名冊路徑 ★請注意前面必須有小r★

projTT = pd.read_excel(regi_path)
projTT


error_check = '程式查無資料，請手動查詢。' in projTT[:13].values
error_check

if error_check == True:
    print('有資料缺失')

In [36]:
projTT.iloc[:,12]

0              計畫詳目
1              報告詳目
2     程式查無資料，請手動查詢。
3              計畫詳目
4              計畫詳目
5              計畫詳目
6              計畫詳目
7              計畫詳目
8              計畫詳目
9              計畫詳目
10             計畫詳目
Name: 網站依據, dtype: object

In [37]:
projTT

,計畫主持人,計畫編號,執行機構,中文計畫名稱,英文計畫名稱,研究領域,研究經費,研究期間,中文關鍵字,英文關鍵字,中文摘要,英文摘要,網站依據
0,巫博瀚,MOST109-2410-H130-032-SS3,銘傳大學教育研究所,台灣國小學生興趣與自尊發展之縱貫性研究： 新舊課綱世代學童之發展趨勢與世代分析,The Longitudinal Analysis of the Development of Interest and Self-Esteem among Taiwanese Primary Students: the Development Trend and Cohort Analysis of Students of Generations Adopting the Old and New Curriculum Guidelines,教育學；,926千元,"['10908 ', ' 11007']",興趣；自尊；成就歸因；序列設計；潛在成長模型,interest； self-esteem； achievement attribution； sequential design； latent growth model,"興趣是個體知識獲取的驅動力，也是促進學生深度學習的關鍵因素。此外，自尊是人類心靈健康的核心要素，更是人類的基本需求。然而，歷次TIMSS的調查均顯示，台灣學童的數學興趣與自信均在全球倒數之列，且低興趣與低信心對學習的危害迄今仍持續發生！目前國內正在推動的12年國民基本教育之課程發展強調培養學生的熱情與興趣，使學生能適性發展、悅納自己、自主學習並展現自信。由於國小學童的興趣與自尊之發展趨勢及其影響因素，迄今尚無實證研究予以探討，此外，為瞭解新舊課綱世代學生在興趣與自尊上的差異，本計劃將透過序列設計，針對不同世代（「九年一貫課綱世代」與「108課綱世代」）國小學生進行為期3年共6波次的重複測量與追蹤調查。透過序列設計，本計畫將能區分發展趨勢與世代差異，不僅能有效了解國小學生數學興趣與自尊的發展趨勢與成長軌跡、釐清成就歸因對興趣與自尊發展情形之影響，亦可進行世代分析，探討新課綱的實施是否能滿足預期目標。在兼顧城鄉差異與樣本異質性下，第一波的小一與小三樣本分別自台北市、新北市及桃園市抽取40班約1,000人作為研究樣本，並以潛在成長模型(latent growth model)分析所蒐集的重複測量資料。","Interest seems to represent a driving force for knowledge gains, and it is also the key factor for promoting students’ further study. In addition, self-esteem is also a core element for human mental health and one of human’s basic needs. However, the surveys of TIMSS across years show that Taiwanese students’ interest and confidence in math are at the bottom of the list, and the lack of interest and confidence is still having some negative impact on students’ learning. The development of the new curriculum for the 12-year Basic Education currently promoted in Taiwan puts much emphasis on cultivating students’ passion for and interest in learning, aiming to enable students to develop according to their own nature, accept themselves, learn by themselves, and show their confidence in themselves. Since there is no empirical research on the development tendency and influencing factors of primary school students’ interest and self-esteem, in order to explore the difference in interest and self-esteem between students of generations adopting the old and new curriculum guidelines, the present studeny will drawn on the method of sequential design to measure and investigate primary school students of different generations of curriculum guidlines (Grade 1-9 Curriulum Guidelines v. 108 Curriulum Guidelines) for six waves in three years. With the use of sequential design, the present project will be able to distinguish between development tendency and generational difference. This allows us to effectively examine Taiwanese primary school students’s development tendency and growth trajectory and clarify the influence of achievement attribution on the development of interest and self-esteem. A generational analysis can also be conducted to determine if the implementation of the new curriculum guidelines helps to achieve the expected objectives. In order to address the difference between urban and rural areas and ensure the hetergeneity of the samples, the first wave of the grade-1 and grade-3 samples will consist of approximately 1000 students from 40 classes in primary schools in Taipei, New Taipei and Taoyuan, respectively. The collected data will be analyzed using the latent growth model.",計畫詳目
1,田雲生；張金元；陳令錫；沈原民；李安心 Yun-Sheng Tien；Chin-Yuan Chang；Ling-Hsi Chen；Yuan-Min Shen；An-Sen Li,101農科-4.1.1-中-D2,行政院農業委員會臺中區農業改良場作物環境課,高效率節能型溫室環控系統之研究,Study on High-Efficiency Energy-Saving Environmental Controlled System in Greenhouse,農業機械；園藝；,2000千元,"['10101 ', ' 10112']",無線監測；高效率；環控；節能,Remote Sensing；High-efficiency；Environmental Control；Energy-saving,NaN,NaN,報告詳目
2,林文皓,NaN,NaN,雞雞太小,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,程式查無資料，請手動查詢。
3,繆敏志,MOST109-2410-H034-021-SSS,中國文化大學國際企業管理學系,跨層次觀點下佛系行為與工作績效之關係：僕人式領導與分配正義之調節角色,A Cross-Level Examination on Relationships between Buddhist Behavior and Job Performance : Moderating Role of Servant Leadership and Distributive Justice,管理科學；,364千元,"['10908 ', ' 11007']",佛系行為；僕人式領導；分配正義；工作績效；層級線性模式,Buddhist behavior；